### Общее задание
Необходимо написать программу, которая будет получать данные от пользователя, вносить их в базу, осуществлять поиск по базе по запросу пользователя.

Хранение каких-либо констант и справочников в коде ЗАПРЕЩЕНО!

### Задание
У нас есть чат. Необходимо хранить профили пользователей, сообщения в общий чат и сообщения в личку (от кого кому).

In [2]:
import os
import sqlite3
import choice

In [3]:
def connect(dbfile):
    conn = sqlite3.connect(dbfile)
    c = conn.cursor()
    return c, conn

In [4]:
def create_db():
    dbfile = input('Type the name of the database to create (without a filename extension): ') + '.db'
    has_db = os.path.exists(dbfile)
    if has_db: 
        print('The database with this name has already existed.')
        return create_db()
    c, conn = connect(dbfile)
    print('The database "{}" is created successfully.'.format(dbfile))
    c.close()

# Создаем бд

In [5]:
create_db()

Type the name of the database to create (without a filename extension): dialogs
The database "dialogs.db" is created successfully.


In [6]:
def create_table():
    dbfile = input('Type the name of the database where you want to create a table\
    (without filename extension): ') + '.db'
    if not os.path.isfile(dbfile):
        print('This database does not exist. Try once again')
        create_table()
    else:
        c, conn = connect(dbfile)
        table_name = input('Type the name of the table to create: ').upper()
        columns = input('Type the names of the columns and their types as in the EXAMPLE:\
        \n"name1 text, name2 text, name3 integer"\n') 
        c.execute('CREATE TABLE {} (generated_id INTEGER PRIMARY KEY, {})'.format(table_name, columns))
        print('The table "{}" in the database "{}" is created successfully.'.format(table_name, dbfile))
        conn.commit()
        conn.close()

# Создаём таблицы

### TABLES:

1) USERS: user text, url text

2) GENERAL_CHAT: massege text, who integer, data datetime

3) PERSONAL_CHAT: massege text, who integer, whom integer, data datetime

In [7]:
create_table()

Type the name of the database where you want to create a table    (without filename extension): dialogs
Type the name of the table to create: USERS
Type the names of the columns and their types as in the EXAMPLE:        
"name1 text, name2 text, name3 integer"
user text, url text
The table "USERS" in the database "dialogs.db" is created successfully.


In [8]:
create_table()

Type the name of the database where you want to create a table    (without filename extension): dialogs
Type the name of the table to create: GENERAL_CHAT
Type the names of the columns and their types as in the EXAMPLE:        
"name1 text, name2 text, name3 integer"
massege text, who integer, data datetime
The table "GENERAL_CHAT" in the database "dialogs.db" is created successfully.


In [36]:
create_table()

Type the name of the database where you want to create a table    (without filename extension): dialogs
Type the name of the table to create: PERSONAL_CHAT
Type the names of the columns and their types as in the EXAMPLE:        
"name1 text, name2 text, name3 integer"
massege text, who integer, whom integer, data datetime
The table "PERSONAL_CHAT" in the database "dialogs.db" is created successfully.


In [10]:
def parsing(answer):
    parsed_answer = []
    for row in answer.split('; '):
        parsed_answer.append(row.split(', '))
    return parsed_answer

In [37]:
def fill_table_USERS(dbfile):
    c, conn = connect(dbfile)
    info = input('Type the information as in the EXAMPLE:\
    \n"user1, url1; user2, url2"\n')
    users = [user[0] for user in c.execute('''SELECT user FROM USERS''')]
    urls = [url[0] for url in c.execute('''SELECT url FROM USERS''')]
    parsed_info  = parsing(info)
    
    # условимся, что не может быть у юзеров одинаковые имена и url
    exist = False
    for row in parsed_info:
        if row[0] in users or row[1] in urls:
            exist = True

    # пользователь мог ввести в запросе одинаковые имена и url
    typed_users = [u[0] for u in parsed_info]
    typed_urls = [r[1] for r in parsed_info]
    if len(set(typed_users)) != len(typed_users):
        exist = True
    if len(set(typed_urls)) != len(typed_urls):
        exist = True
    if exist:
        print('User or URL already exists in this database or you typed the same names! Try again.')
        fill_table_USERS(dbfile)
    else:
        c.executemany('''INSERT INTO USERS (generated_id, user, url) VALUES (null, ?, ?)''', parsed_info)
        conn.commit()
        conn.close()
        print('Information is filled successfully.')

from datetime import datetime

def fill_table_GENERAL_CHAT(dbfile):
    c, conn = connect(dbfile)
    info = input('Type the information as in the EXAMPLE:\
    \n"massege1, who1, date1; massege2, who2"\n')
    parsed_info  = parsing(info)
    for each in parsed_info:
        each.append(datetime.now().strftime("%B %d, %Y %I:%M%p"))
    #print(parsed_info)
    c.executemany('''INSERT INTO GENERAL_CHAT (generated_id, massege, who, data) VALUES (null, ?, ?, ?)''', parsed_info)
    conn.commit()
    conn.close()
    print('Information is filled successfully.')

def fill_table_PERSONAL_CHAT(dbfile):
    c, conn = connect(dbfile)
    info = input('Type the information as in the EXAMPLE:\
    \n"massege1, who1, whom1; massege2, who2, whom"\n')
    parsed_info  = parsing(info)
    for each in parsed_info:
        each.append(datetime.now().strftime("%B %d, %Y %I:%M%p"))
    c.executemany('''INSERT INTO PERSONAL_CHAT (generated_id, massege, who, whom, data) VALUES (null, ?, ?, ?, ?)''', parsed_info)
    conn.commit()
    conn.close()
    print('Information is filled successfully.')

In [38]:
def fill_table():
    dbfile = input('Type the name of the database where you want to fill a table\
    (without filename extension): ') + '.db'
    if not os.path.isfile(dbfile):
        print('This database does not exist. Try once again')
        fill_table()
    c, conn = connect(dbfile)
    c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [table[0] for table in c.fetchall()]
    tables_str = '\n*'.join(tables)
    table_to_fill = input('Type the name of the table which you want to fill (from the list):\n*' + tables_str + '\n').upper()
    tables = ['USERS', 'GENERAL_CHAT', 'PERSONAL_CHAT']
    ind = tables.index(table_to_fill.upper())
    if ind == 0:
        fill_table_USERS(dbfile)
    elif ind == 1:
        fill_table_GENERAL_CHAT(dbfile)
    else:
        fill_table_PERSONAL_CHAT(dbfile)

# Заполняем таблицы

In [16]:
fill_table()

Type the name of the database where you want to fill a table    (without filename extension): dialogs
Type the name of the table which you want to fill (from the list):
*USERS
*GENERAL_CHAT
*PERSONAL_CHAT
USERS
Type the information as in the EXAMPLE:    
"user1, url1; user2, url2"
user1, url1; user2, url2; user3, url3; user4, url4
Information is filled successfully.


In [34]:
fill_table()

Type the name of the database where you want to fill a table    (without filename extension): dialogs
Type the name of the table which you want to fill (from the list):
*USERS
*GENERAL_CHAT
*PERSONAL_CHAT
GENERAL_CHAT
Type the information as in the EXAMPLE:    
"massege1, who1, date1; massege2, who2"
massege1, user1; massege2, user1; massege3, user2
Information is filled successfully.


In [39]:
fill_table()

Type the name of the database where you want to fill a table    (without filename extension): dialogs
Type the name of the table which you want to fill (from the list):
*USERS
*GENERAL_CHAT
*PERSONAL_CHAT
PERSONAL_CHAT
Type the information as in the EXAMPLE:    
"massege1, who1, whom1; massege2, who2, whom"
massege1, user1, user2; massege2, user1, user3; massege5, user2, user3; massege3, user2, user3;  massege4, user2, user3
Information is filled successfully.


### Что пользователь может искать:

1) Количество уникальных пользователей в бд (и кто они)

2) Кол-во пользователей, которые не общаются в общем чате (и кто они)

3) Кол-во пользователей, который общаются только в общем чате (и кто они)

4) Сортировать сообщения в общем чате по дате

5) Кому что пишет отдельный пользователь (в личку)

6) Диалог между двумя пользователями

7) Сортировать по алфавиту всех пользователей

In [68]:
queries = ['Количество уникальных пользователей в бд (и кто они)',
           'Кол-во пользователей, которые не общаются в общем чате (и кто они)',
           'Кол-во пользователей, который общаются только в общем чате (и кто они)',
           'Сортировать сообщения в общем чате по дате',
           'Кому что пишет отдельный пользователь',
           'Диалог между двумя пользователями',
           'Сортировать по алфавиту всех пользователей']

In [69]:
def first(dbfile):
    c, conn = connect(dbfile)
    c.execute("SELECT COUNT(*) FROM (SELECT user FROM USERS) UNION SELECT user FROM USERS")# первый элемент - кол-во
    result = [user[0] for user in c.fetchall()]
    print(result)

In [70]:
def second(dbfile):
    c, conn = connect(dbfile)
    c.execute("SELECT * FROM USERS WHERE NOT EXISTS (SELECT * FROM GENERAL_CHAT WHERE USERS.user = GENERAL_CHAT.who)")
    result = [user[1] for user in c.fetchall()]
    print(result)
    c, conn = connect(dbfile)
    c.execute("SELECT COUNT(*) FROM (SELECT * FROM USERS WHERE NOT EXISTS (SELECT * FROM GENERAL_CHAT WHERE USERS.user = GENERAL_CHAT.who))")
    result = c.fetchall()[0][0]
    print(result)

In [71]:
def third(dbfile):
    c, conn = connect(dbfile)
    c.execute("SELECT * FROM GENERAL_CHAT WHERE NOT EXISTS (SELECT * FROM PERSONAL_CHAT WHERE GENERAL_CHAT.who = PERSONAL_CHAT.who)")
    result = [user[1] for user in c.fetchall()]
    print(result)
    c, conn = connect(dbfile)
    c.execute("SELECT COUNT(*) FROM (SELECT * FROM GENERAL_CHAT WHERE NOT EXISTS (SELECT * FROM PERSONAL_CHAT WHERE GENERAL_CHAT.who = PERSONAL_CHAT.who))")
    result = c.fetchall()[0][0]
    print(result)

In [77]:
def forth(dbfile):
    c, conn = connect(dbfile)
    c.execute("SELECT * FROM GENERAl_CHAT ORDER BY data")# первый элемент - кол-во
    result = c.fetchall()
    print(result)

In [73]:
def fifth(dbfile):
    user = input('Type the name the user in whom you are interested in:')
    c, conn = connect(dbfile)
    c.execute("SELECT * FROM PERSONAL_CHAT WHERE who='{}'".format(user))
    result = [[user[1], user[3]] for user in c.fetchall()]
    print(result)

In [74]:
def sixth(dbfile):
    users = input('Type the names of the usera in whom you are interested in as in the EXAMPLE:\
    \n"user1, user2"\n')
    parsed_users = parsing(users)[0]
    print(parsed_users)
    c, conn = connect(dbfile)
    c.execute("SELECT * FROM PERSONAL_CHAT WHERE who='{}' AND whom='{}' UNION SELECT * FROM PERSONAL_CHAT WHERE who='{}' AND whom='{}'".format(parsed_users[1], parsed_users[0], parsed_users[0], parsed_users[1]))
    result = [user for user in c.fetchall()]
    print(result)

In [75]:
def seventh(dbfile):
    c, conn = connect(dbfile)
    c.execute("SELECT user, url FROM USERS ORDER BY user")# первый элемент - кол-во
    result = [user[0] for user in c.fetchall()]
    print(result)

In [49]:
def search():
    dbfile = input('Type the name of the database where you want to search\
    (without filename extension): ') + '.db'
    if not os.path.isfile(dbfile):
        print('This database does not exist. Try once again')
        search()
    query = choice.Menu(queries).ask()
    ind = queries.index(query)
    if ind == 0:
        first(dbfile)
    if ind == 1:
        second(dbfile)
    if ind == 2:
        third(dbfile)
    if ind == 3:
        forth(dbfile)
    if ind == 4:
        fifth(dbfile)
    if ind == 5:
        sixth(dbfile)
    if ind == 6:
        seventh(dbfile)
    if ind == 8:
        eight(dbfile)

# Ищем по бд

In [50]:
search()

Type the name of the database where you want to search    (without filename extension): dialogs
Make a choice:
 0: Количество уникальных пользователей в бд (и кто они)
 1: Кол-во пользователей, которые не общаются в общем чате (и кто они)
 2: Кол-во пользователей, который общаются только в общем чате (и кто они)
 3: Кол-во пользователей, которые вообще не общаются (и кто они)
 4: Кому что пишет отдельный пользователь
 5: Диалог между двумя пользователями
 6: Сортировать по алфавиту всех пользователей
 7: Сортировать сообщения в общем чате по дате

Enter number or name; return for next page
? 0

[4, 'user1', 'user2', 'user3', 'user4']


In [51]:
search()

Type the name of the database where you want to search    (without filename extension): dialogs
Make a choice:
 0: Количество уникальных пользователей в бд (и кто они)
 1: Кол-во пользователей, которые не общаются в общем чате (и кто они)
 2: Кол-во пользователей, который общаются только в общем чате (и кто они)
 3: Кол-во пользователей, которые вообще не общаются (и кто они)
 4: Кому что пишет отдельный пользователь
 5: Диалог между двумя пользователями
 6: Сортировать по алфавиту всех пользователей
 7: Сортировать сообщения в общем чате по дате

Enter number or name; return for next page
? 1

['user3', 'user4']
2


In [52]:
search()

Type the name of the database where you want to search    (without filename extension): dialogs
Make a choice:
 0: Количество уникальных пользователей в бд (и кто они)
 1: Кол-во пользователей, которые не общаются в общем чате (и кто они)
 2: Кол-во пользователей, который общаются только в общем чате (и кто они)
 3: Кол-во пользователей, которые вообще не общаются (и кто они)
 4: Кому что пишет отдельный пользователь
 5: Диалог между двумя пользователями
 6: Сортировать по алфавиту всех пользователей
 7: Сортировать сообщения в общем чате по дате

Enter number or name; return for next page
? 2

[]
0


In [78]:
search()

Type the name of the database where you want to search    (without filename extension): dialogs
Make a choice:
 0: Количество уникальных пользователей в бд (и кто они)
 1: Кол-во пользователей, которые не общаются в общем чате (и кто они)
 2: Кол-во пользователей, который общаются только в общем чате (и кто они)
 3: Сортировать сообщения в общем чате по дате
 4: Кому что пишет отдельный пользователь
 5: Диалог между двумя пользователями
 6: Сортировать по алфавиту всех пользователей

Enter number or name; return for next page
? 3

[(1, 'massege1', 'user1', 'October 14, 2018 10:56PM'), (2, 'massege2', 'user1', 'October 14, 2018 10:56PM'), (3, 'massege3', 'user2', 'October 14, 2018 10:56PM'), (4, 'massege1', 'user1', 'October 14, 2018 10:56PM'), (5, 'massege2', 'user1', 'October 14, 2018 10:56PM'), (6, 'massege3', 'user2', 'October 14, 2018 10:56PM')]


In [54]:
search()

Type the name of the database where you want to search    (without filename extension): dialogs
Make a choice:
 0: Количество уникальных пользователей в бд (и кто они)
 1: Кол-во пользователей, которые не общаются в общем чате (и кто они)
 2: Кол-во пользователей, который общаются только в общем чате (и кто они)
 3: Кол-во пользователей, которые вообще не общаются (и кто они)
 4: Кому что пишет отдельный пользователь
 5: Диалог между двумя пользователями
 6: Сортировать по алфавиту всех пользователей
 7: Сортировать сообщения в общем чате по дате

Enter number or name; return for next page
? 4

Type the name the user in whom you are interested in:user1
[['massege1', 'user2'], ['massege2', 'user3']]


In [61]:
search()

Type the name of the database where you want to search    (without filename extension): dialogs
Make a choice:
 0: Количество уникальных пользователей в бд (и кто они)
 1: Кол-во пользователей, которые не общаются в общем чате (и кто они)
 2: Кол-во пользователей, который общаются только в общем чате (и кто они)
 3: Кол-во пользователей, которые вообще не общаются (и кто они)
 4: Кому что пишет отдельный пользователь
 5: Диалог между двумя пользователями
 6: Сортировать по алфавиту всех пользователей
 7: Сортировать сообщения в общем чате по дате

Enter number or name; return for next page
? 5

Type the names of the usera in whom you are interested in as in the EXAMPLE:    
"user1, user2"
user1, user2
['user1', 'user2']
[(1, 'massege1', 'user1', 'user2', 'October 14, 2018 11:01PM')]


In [56]:
search()

Type the name of the database where you want to search    (without filename extension): dialogs
Make a choice:
 0: Количество уникальных пользователей в бд (и кто они)
 1: Кол-во пользователей, которые не общаются в общем чате (и кто они)
 2: Кол-во пользователей, который общаются только в общем чате (и кто они)
 3: Кол-во пользователей, которые вообще не общаются (и кто они)
 4: Кому что пишет отдельный пользователь
 5: Диалог между двумя пользователями
 6: Сортировать по алфавиту всех пользователей
 7: Сортировать сообщения в общем чате по дате

Enter number or name; return for next page
? 6

['user1', 'user2', 'user3', 'user4']


In [57]:
search()

Type the name of the database where you want to search    (without filename extension): dialogs
Make a choice:
 0: Количество уникальных пользователей в бд (и кто они)
 1: Кол-во пользователей, которые не общаются в общем чате (и кто они)
 2: Кол-во пользователей, который общаются только в общем чате (и кто они)
 3: Кол-во пользователей, которые вообще не общаются (и кто они)
 4: Кому что пишет отдельный пользователь
 5: Диалог между двумя пользователями
 6: Сортировать по алфавиту всех пользователей
 7: Сортировать сообщения в общем чате по дате

Enter number or name; return for next page
? 7

